In [1]:
from bs4 import BeautifulSoup
import requests
import time
import csv
#import dict_builder as db
import re
import pandas as pd
import requests
import numpy as np
#from __future__ import unicode_literals 
#from bosonnlp import BosonNLP
#nlp = BosonNLP('Yy7Tpb83.27824.Zdrfgp6JtKTf')
import os
import random
import pickle
import nltk
from scipy import spatial
Porter = nltk.stem.PorterStemmer()

In [115]:
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
#FTzh = FastText.load('FTzh.model')
#FTen = FastText.load('FTen.model')
CBzh = Word2Vec.load('CBzh0505.model')
CBen = Word2Vec.load('CBen0505.model')
#SGzh = Word2Vec.load('SGzh.model')
#SGen = Word2Vec.load('SGen.model')

In [43]:
with open('FT_trans_matrix.pickle', 'rb') as f:
    FastText = pickle.load(f)
with open('CB_trans_matrix.pickle', 'rb') as f:
    Cbow = pickle.load(f)
with open('SG_trans_matrix.pickle', 'rb') as f:
    Skipgram = pickle.load(f)

In [116]:
with open('0505_trans_matrix.pickle','rb') as f:
    Cbow = pickle.load(f)

In [4]:
def readfile(filename,method='utf-8'):
    rawlist = []
    with open(filename,'r+', encoding=method) as csvfile:
        rows = csv.reader(csvfile)
        for row in rows:
            rawlist.append(row)
    return rawlist

def writefile(filename,inputlist,method='utf-8'):
    with open(filename,'a+', encoding=method) as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        for row in inputlist:
            writer.writerow(row)

In [5]:
def regex(a):
    reg = []
    #a = re.sub('[a-zA-Z]','',a)
    a = re.sub('[,、：；！？，。－"\']','#',a)
    a = re.sub('[「」(){}:;!?]','',a)
    a = re.sub('\n','',a)
    #a = re.sub('[(),{}\u3000\*\|=-\[\]\n.]','',a)
    #a = re.sub(u"\\（.*?\\）|\\『.*?』|\\「.*?」|\\〔.*?〕|\\[.*?]|\\〈.*?〉", '', a)
    reg = a.split('#')
    return reg

def en_regex(a):
    reg = []
    #a = re.sub('[a-zA-Z]','',a)
    a = re.sub('[{}.:;!?：；！？，。－"\']','#',a)
    a = re.sub('[、「」()]','',a)
    a = re.sub(',',' ',a)
    a = re.sub('\n','',a)
    #a = re.sub('[(),{}\u3000\*\|=-\[\]\n.]','',a)
    #a = re.sub(u"\\（.*?\\）|\\『.*?』|\\「.*?」|\\〔.*?〕|\\[.*?]|\\〈.*?〉", '', a)
    reg = a.split('#')
    return reg

def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

# 從這裡開始跑

In [55]:
with open('airiti_english_corpus.pickle','rb') as f:
    en_data = pickle.load(f)
with open('airiti_chinese_corpus.pickle','rb') as f:
    zh_data = pickle.load(f)
len(zh_data)

25695

In [56]:
"""
with open('./CUTs/paper_chcutBA.pickle','rb') as f:
    zh_data = pickle.load(f)
    print(len(zh_data))
with open('./CUTs/paper_chcutEE.pickle','rb') as f:
    tmp = pickle.load(f)
    zh_data = zh_data+tmp
    print(len(zh_data))
with open('./CUTs/paper_chcutMED.pickle','rb') as f:
    tmp = pickle.load(f)
    zh_data = zh_data+tmp
    print(len(zh_data))
with open('./CUTs/paper_chcutENG.pickle','rb') as f:
    tmp = pickle.load(f)
    zh_data = zh_data+tmp
    print(len(zh_data))
with open('./CUTs/paper_encutBA.pickle','rb') as f:
    en_data = pickle.load(f)
    print(len(en_data))
with open('./CUTs/paper_encutEE.pickle','rb') as f:
    tmp = pickle.load(f)
    en_data = en_data+tmp
    print(len(en_data))
with open('./CUTs/paper_encutMED.pickle','rb') as f:
    tmp = pickle.load(f)
    en_data = en_data+tmp
    print(len(en_data))
with open('./CUTs/paper_encutENG.pickle','rb') as f:
    tmp = pickle.load(f)
    en_data = en_data+tmp
    print(len(en_data))
"""
print('no longer need this')

no longer need this


In [7]:
evalset = readfile('Bert_evaluation.csv')

## Threshold 根據 eval set 一千句平均的值

In [120]:
match = []
match2 = []
for i in range(0,1000):
    match.append(evalset[i][0].split())
    match2.append(evalset[i][2].split('#'))

In [121]:
bertvectors = CBen
chinese_bertvectors = CBzh
transmatrix = Cbow
sum1 = 0
for i in range(0,1000):
    sum1+=bert_matrix(match2[i],match[i])
sum1 = sum1/1000
print(sum1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


0.4279196787495523


## 簡單的5篇文章實驗

In [151]:
for line in zh_data[1]:
    print(line)

[]
['自動', '駕駛', '車輛', '是', '一', '種', '新興', '技術']
['它', '有', '許多', '不同', '類型', '任務', 'task', '的', '需求']
['包括', '低', '延遲', '的', '運算', '任務', '和', '資源', '密集型', '的', '運算', '任務']
['由於', '車輛', '的', '計算', '能力', '和', '儲存', '容量', '有限']
['如何', '服務', '如此', '大量', '的', '任務', '已', '成為', '車用', '行動', '通訊', '網路', '中', '的', '嚴峻', '挑戰']
['因此', '本', '研究', '將', '使用', '雲端', '運算', '系統', '來', '克服', '車輛', '自身', '資源', '有限', '的', '問題']
['並', '利用', '高速公路', '有', '固定', '路線', '的', '優點']


In [152]:
for line in en_data[1]:
    print(line)

['self-driving', 'vehicle', 'is', 'an', 'emerging', 'technology', 'which', 'request', 'many', 'different', 'types', 'of', 'tasks', 'including', 'low-latency', 'computation', 'tasks', 'and', 'resource', 'intensive', 'computation', 'tasks']
['due', 'to', 'the', 'limited', 'computational', 'capabilities', 'and', 'storage', 'capacity', 'of', 'vehicles', 'serving', 'such', 'a', 'large', 'number', 'of', 'tasks', 'has', 'become', 'a', 'serious', 'challenge', 'in', 'the', 'vehicular', 'network']
['therefore', 'this', 'study', 'will', 'use', 'mobile', 'cloud', 'computing', 'systems', 'to', 'overcome', 'the', 'problem', 'of', 'the', 'limited', 'resources', 'in', 'vehicles']
['by', 'taking', 'the', 'advantages', 'of', 'the', 'fixed', 'route', 'of', 'highway', 'the', 'direction', 'and', 'the', 'speed', 'of', 'vehicles', 'can', 'be', 'more', 'predictable']
[]


In [221]:
op = []
for i in range(0,10):
    op.append(revised_greedy(chlist=zh_data[i][1:],enlist=en_data[i],threshold=0.4,method='matrix'))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


window up into wash
window up into wash
window up into wash
No chinese so into wash
No chinese so into wash
No chinese so into wash
No chinese so into wash
window up into wash
17 ['自動', '駕駛', '車輛', '是', '一', '種', '新興', '技術', '它', '有', '許多', '不同', '類型', '任務', 'task', '的', '需求']
22 ['self-driving', 'vehicle', 'is', 'an', 'emerging', 'technology', 'which', 'request', 'many', 'different', 'types', 'of', 'tasks', 'including', 'low-latency', 'computation', 'tasks', 'and', 'resource', 'intensive', 'computation', 'tasks']
Combination of 0+1
R:0.410251
C:自動駕駛車輛是一種新興技術它有許多不同類型任務task的需求
E: self-driving vehicle is an emerging technology which request many different types of tasks including low-latency computation tasks and resource intensive computation tasks

window up into wash
21 ['包括', '低', '延遲', '的', '運算', '任務', '和', '資源', '密集型', '的', '運算', '任務', '由於', '車輛', '的', '計算', '能力', '和', '儲存', '容量', '有限']
27 ['due', 'to', 'the', 'limited', 'computational', 'capabilities', 'and', 'storage', 'capacity'

No chinese so into wash
No chinese so into wash
No chinese so into wash


In [220]:
counte = 0
for line in op:
    for dope in line:
        print(counte+1,'C:',dope[0],'\n',' E:',dope[1].strip(),'\n')
        counte+=1

1 C: 自動駕駛車輛是一種新興技術它有許多不同類型任務task的需求 
  E: self-driving vehicle is an emerging technology which request many different types of tasks including low-latency computation tasks and resource intensive computation tasks 

2 C: 包括低延遲的運算任務和資源密集型的運算任務由於車輛的計算能力和儲存容量有限 
  E: due to the limited computational capabilities and storage capacity of vehicles serving such a large number of tasks has become a serious challenge in the vehicular network 

3 C: 因此本研究將使用雲端運算系統來克服車輛自身資源有限的問題 
  E: therefore this study will use mobile cloud computing systems to overcome the problem of the limited resources in vehicles 

4 C: 過去人們自閱讀相關的期刊論文專利或報導等資料中歸納得出研究領域的主題變化 
  E: in the past people consumed related research thesis journal papers or patent specifications to conclude the research trend of a topic area 

5 C: 隨着智慧型手機與無線網路的普及行動支付在世界各國迅速發展 
  E: as the popularity of smart phones and wireless communication networks raises mobile payments have been rapidly developed around the world 

6 C: 雖然台灣是使用智慧型手機人口最多的國家但是消費

In [207]:
for line in ansset:
    print(ansset)

[['自動駕駛車輛是一種新興技術它有許多不同類型任務task的需求', ' self-driving vehicle is an emerging technology which request many different types of tasks including low-latency computation tasks and resource intensive computation tasks'], ['包括低延遲的運算任務和資源密集型的運算任務由於車輛的計算能力和儲存容量有限', ' due to the limited computational capabilities and storage capacity of vehicles serving such a large number of tasks has become a serious challenge in the vehicular network'], ['因此本研究將使用雲端運算系統來克服車輛自身資源有限的問題', ' therefore this study will use mobile cloud computing systems to overcome the problem of the limited resources in vehicles'], ['過去人們自閱讀相關的期刊論文專利或報導等資料中歸納得出研究領域的主題變化', ' in the past people consumed related research thesis journal papers or patent specifications to conclude the research trend of a topic area'], ['隨着智慧型手機與無線網路的普及行動支付在世界各國迅速發展', ' as the popularity of smart phones and wireless communication networks raises mobile payments have been rapidly developed around the world'], ['雖然台灣是使用智慧型手機人口最多的國家但是消費者對行動支付的使用率並不高落後於南韓', '

# 開始用的到

In [9]:
with open('lucas_vectors.pickle', 'rb') as file:
    bertvectors = pickle.load(file)
len(bertvectors)

172357

In [57]:
with open('cleaner_dict.pickle','rb') as p:
    adict = pickle.load(p)
len(adict)

50009

# bert 相似度演算法

In [48]:
def bert_calculator(cs,es):
    s2s_rate = 0
    #penalty = 1 - (np.abs(len(cs)-len(es))/(len(cs)+len(es)))
    #oov = np.ones(768)
    oov = np.ones(200)
    for i in range(0,len(cs)):
        try:
            translations = adict[cs[i]]
        except:
            translations = [cs[i]]
        tmpcossim = 0
        #print('*',translations)
        for trans in translations:
            #print(trans)
            if not trans == '':
                try:
                    bert_ch = bertvectors[trans]
                except:
                    bert_ch = oov
                for en_word in es:
                    try:
                        bert_en = bertvectors[en_word]
                    except:
                        bert_en = oov
                    try:
                        teststat = 1-spatial.distance.cosine(bert_en,bert_ch)
                    except:
                        teststat = 0
                    #print(trans,'/',en_word,teststat)
                    if (teststat > tmpcossim):
                        tmpcossim = teststat
        s2s_rate = s2s_rate + tmpcossim
    if(len(cs))==0:
        return 0
    #s2s_rate = float(s2s_rate/len(cs))*penalty 
    s2s_rate = float(s2s_rate/len(cs))
    return(s2s_rate)

## 轉換矩陣

In [132]:
def bert_matrix(cs,es,with_penalty = False):
    s2s_rate = 0
    no_test = False
    no_such_ch_word = False
    negate = 0
    
    #print(len(cs),cs)
    for i in range(0,len(cs)):
        ch_word = cs[i]
        tmpcossim = 0
        ####
        if not ch_word == '':
            try:
                bert_ch = chinese_bertvectors[ch_word]
                bert_ch = np.dot(bert_ch,transmatrix)
            except:
                #bert_ch = bc.encode([ch_word])
                no_such_ch_word = True
        else:
            no_such_ch_word = True
        ####
        ####
        if not no_such_ch_word:
            for en_word in es:
                try:
                    bert_en = bertvectors[en_word.lower()]
                except:
                    no_test = True

                if no_test:
                    teststat = 0
                else:
                    teststat = 1-spatial.distance.cosine(bert_en,bert_ch)
                no_test = False
                if (teststat > tmpcossim):
                    tmpcossim = teststat
            s2s_rate = s2s_rate + tmpcossim
        else:
            negate = negate+1
            no_such_ch_word = False
        ####    
    if(len(cs))==0:
        return 0
    
    
    if with_penalty:
        penalty = 1 - (np.abs(len(cs)-negate-len(es))/(len(cs)-negate+len(es)))
        #print('penalty:',penalty)
        s2s_rate = float(s2s_rate/(len(cs)-negate))*penalty
    else:
        s2s_rate = float(s2s_rate/(len(cs)-negate))
    
    return(s2s_rate)

In [67]:
def choose_your_algo(c,e,method='bert',penalty=False):
    rate = 0
    if method=='rank':
        rate = s2s_calculator(c,e)
    elif method=='average':
        rate = another_s2s_calculator(c,e,chdic,endic)
    elif method=='bert':
        rate = bert_calculator(c,e)
    elif method=='matrix':
        rate = bert_matrix(c,e,penalty)    
    return rate

In [12]:
def word_output_combination(c,e,i,k):
    ctmp = ''
    etmp = ''
    for word in c:
        ctmp = ctmp + word
    for word in e:
        etmp = etmp+' '+word
    comber = str(i)
    for ktop in range(1,k):
        comber = comber +'+'+ str(i+ktop)
    return ctmp,etmp,comber

In [13]:
def reset_variables(i,k,j):
    igate = i+k
    jgate = j+1
    ceiling = 0
    rate = 0
    return igate,jgate,ceiling,rate

In [33]:
def greedy_operations(ceiling,tmp,en,i,k,j):
    ctmp,etmp,combstr = word_output_combination(tmp,en,i,k)
    print(len(tmp),tmp)
    print(len(en),en)
    if len(combstr)>1:
        print('Combination of {0}'.format(combstr))
    print('R:{0:4f}\nC:{1}\nE:{2}\n'.format(ceiling,ctmp,etmp))
    igate,jgate,ceiling,rate = reset_variables(i,k,j)
    return ctmp,etmp,igate,jgate,ceiling,rate

In [201]:
def elimination(bucket,en,method):
    result = []
    ch = []
    #print('elimination')
    #print('bucket:',len(bucket))
    for i in range(len(bucket),0,-1):
        for x in range(0,i):
            ch = ch+bucket[x]
        rate = choose_your_algo(ch,en,method,penalty=True)
        #print(ch,':\n',rate)
        result.append([x,rate])
        ch = []
    result = sorted(result,key=lambda l:l[1], reverse=True)
    #print('elimination result:',result,'\n')
    return result[0][0],result[0][1]

In [176]:
elimination(ch_template,en_template[0],method = 'matrix')

elimination
['本', '論文', '主要', '討論', '機器', '學習', '機器人', '在', '現代', '對', '人類', '很', '重要', '同時', '電腦', '也', '很', '重要', '我', '喜歡', '論文'] : 0.15629719360069974
['本', '論文', '主要', '討論', '機器', '學習', '機器人', '在', '現代', '對', '人類', '很', '重要', '同時', '電腦', '也', '很', '重要'] : 0.16936288019870652
['本', '論文', '主要', '討論', '機器', '學習', '機器人', '在', '現代', '對', '人類', '很', '重要'] : 0.23519576357510588
['本', '論文', '主要', '討論', '機器', '學習', '機器人', '在', '現代'] : 0.36619439111148966
['本', '論文', '主要', '討論', '機器', '學習'] : 0.5750354533308054


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(0, 0.5750354533308054)

## greedy linear 演算法

In [101]:
def greedy(chlist,enlist,method='bert',threshold=0.85):
    igate = 0
    jgate = 0
    ceiling = 0
    rate = 0
    output = []
    ctmp = ''
    etmp = ''
    
    for i in range(0,len(chlist)):
        tmp = ''
        if i>=igate:
            for j in range(jgate,i+5):
                if j<len(enlist) and j>=0:
                    rate = choose_your_algo(chlist[i],enlist[j],method)    
                    ceiling = rate
                    #print(i,j,rate)
                    if rate>threshold:
                        tmp = chlist[i]
                        #print(tmp)
                        for k in range(1,5):
                            if (i+k)<len(chlist):
                                rate = choose_your_algo((tmp+chlist[i+k]),enlist[j],method)
                                if rate > ceiling:
                                    tmp = tmp + chlist[i+k]
                                    ceiling = rate
                                    #print(tmp)
                                    if k==4:
                                    #Window用完進入洗選階段
                                        ctmp,etmp,igate,jgate,ceiling,rate = greedy_operations(ceiling,
                                                                                           tmp,enlist[j],i,k,j)
                                        output.append([ctmp,etmp])
                                        break
                                else:
                                #結果沒變好直接進入洗選階段
                                    ctmp,etmp,igate,jgate,ceiling,rate = greedy_operations(ceiling,
                                                                                           tmp,enlist[j],i,k,j)
                                    output.append([ctmp,etmp])
                                    break
                            else:
                            #中文用完進入洗選階段
                                ctmp,etmp,igate,jgate,ceiling,rate = greedy_operations(ceiling,
                                                                                           tmp,enlist[j],i,k,j)
                                output.append([ctmp,etmp])
                                break
                        break
    return(output)

## 修正後句子嫁接法 先在沒有距離懲罰的情況加到最長，再在有距離懲罰的情況慢慢砍

In [199]:
def revised_greedy(chlist,enlist,method='matrix',threshold=0.45):
    igate = 0
    jgate = 0
    ceiling = 0
    rate = 0
    output = []
    ctmp = ''
    etmp = ''
    softshold = threshold/3
    bucket = []
    
    for i in range(0,len(chlist)):
        tmp = []
        if i>=igate:
            for j in range(jgate,i+5):
                if j<len(enlist) and j>=0:
                    
                    rate = choose_your_algo(chlist[i],enlist[j],method,penalty=False)    
                    ceiling = rate
                    #print(chlist[i],enlist[j],'\n',i,'',j,'initial ceil:',ceiling,'\n')
                    
                    if rate>softshold:
                        bucket.append(chlist[i])
                        ###
                        for k in range(1,5):
                            if (i+k)<len(chlist):
                                tmp = []
                                for sen in bucket:
                                    tmp = tmp + sen
                                rate = choose_your_algo((tmp+chlist[i+k]),enlist[j],method)
                                
                                #print(tmp+chlist[i+k],enlist[j],'\n',i+k,'',j,'performance:',rate,'\n')
                                
                                bucket.append(chlist[i+k])
                                if rate > ceiling:
                                    ceiling = rate
                                if k==4:
                                #Window用完進入洗選階段
                                    print('window up into wash')
                                    bpos,hard_ceiling = elimination(bucket,enlist[j],method)
                                    if hard_ceiling > threshold:
                                        tmp = []
                                        for pos in range(0,bpos+1):
                                            tmp = tmp + bucket[pos]
                                        ctmp,etmp,igate,jgate,ceiling,rate = greedy_operations(hard_ceiling,
                                                                                               tmp,enlist[j],i,bpos+1,j)
                                        output.append([ctmp,etmp])
                                    else:
                                        igate = igate+1
                                    break
                                elif rate < (ceiling*0.8):
                                #結果變糟太多直接進洗選階段
                                    print(rate,'<',ceiling*0.8,'into wash')
                                    bpos,hard_ceiling = elimination(bucket,enlist[j],method)
                                    if hard_ceiling > threshold:
                                        tmp = []
                                        for pos in range(0,bpos+1):
                                            tmp = tmp + bucket[pos]
                                        ctmp,etmp,igate,jgate,ceiling,rate = greedy_operations(hard_ceiling,
                                                                                            tmp,enlist[j],i,bpos+1,j)
                                        output.append([ctmp,etmp])
                                    else:
                                        igate = igate+1
                                    break
                            else:
                            #中文用完進入洗選階段
                                print('No chinese so into wash')
                                bpos,hard_ceiling = elimination(bucket,enlist[j],method)
                                if hard_ceiling > threshold:
                                    tmp = []
                                    for pos in range(0,bpos+1):
                                        tmp = tmp + bucket[pos]
                                    ctmp,etmp,igate,jgate,ceiling,rate = greedy_operations(hard_ceiling,
                                                                                        tmp,enlist[j],i,bpos+1,j)
                                    output.append([ctmp,etmp])
                                else:
                                    igate = igate+1
                                break
                        ###
                        bucket = []
                        break
    return(output)

In [182]:
def extract_pairs(alignset):
    try1 = []
    for line in alignset:
        for pair in line:
            try1.append(pair)
    return try1

## 0505實驗

In [203]:
revised_greedy(ch_template,en_template,method='matrix',threshold=0.4)

0.45774298888936205 < 0.46002836266464436 into wash
6 ['本', '論文', '主要', '討論', '機器', '學習']
6 ['This', 'paper', 'mainly', 'discuss', 'machine', 'learning']
R:0.575035
C:本論文主要討論機器學習
E: This paper mainly discuss machine learning

No chinese so into wash
12 ['機器人', '在', '現代', '對', '人類', '很', '重要', '同時', '電腦', '也', '很', '重要']
10 ['Robots', 'are', 'important', 'to', 'people', 'nowadays', 'and', 'so', 'do', 'computers']
Combination of 1+2+3
R:0.417417
C:機器人在現代對人類很重要同時電腦也很重要
E: Robots are important to people nowadays and so do computers

No chinese so into wash
3 ['我', '喜歡', '論文']
3 ['I', 'like', 'paper']
R:0.511665
C:我喜歡論文
E: I like paper



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[['本論文主要討論機器學習', ' This paper mainly discuss machine learning'],
 ['機器人在現代對人類很重要同時電腦也很重要',
  ' Robots are important to people nowadays and so do computers'],
 ['我喜歡論文', ' I like paper']]

# 0415實驗

In [62]:
greedy(ch_template,en_template,'matrix',threshold=0)

R:0.327792
C:本論文主要討論機器學習
E: This paper mainly discuss machine learning

R:0.325079
C:機器人在現代
E: Robots are important to people nowadays and so do computers

R:0.065561
C:對人類很重要
E: I like paper



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[['本論文主要討論機器學習', ' This paper mainly discuss machine learning'],
 ['機器人在現代', ' Robots are important to people nowadays and so do computers'],
 ['對人類很重要', ' I like paper']]

# 認真做Alignment

In [14]:
aligned = []
for i in range(0,len(chcut)):
    #print('-----Comparing set {0} using bert-----'.format(i))
    print(i,len(chcut[i]),len(encut[i]))
    aligned.append(greedy(chcut[i],encut[i],method='bert',threshold=0.85))
with open('Paper_Dmpairs_ENG.pickle','wb') as f:
    pickle.dump(aligned,f)

0 8 3
1 4 2
2 6 3
3 21 13
4 36 16
5 27 13
6 10 9
7 14 6
8 11 8
9 9 7
10 27 8
11 8 10
12 17 11
13 3 4
14 5 10
15 17 9
16 11 7
17 7 3
18 25 16
19 10 6
20 6 7
21 21 11
22 14 10
23 14 7
24 11 8
25 11 7
26 16 2
27 10 6
28 4 4
29 3 4
30 31 10
31 15 9
32 6 5
33 19 2
34 10 9
35 10 5
36 9 6
37 23 14
38 40 11
39 16 6
40 33 21
41 4 3
42 0 1
43 18 12
44 12 8
45 4 5
46 5 3
47 0 1
48 12 2
49 24 13
50 32 17
51 15 8
52 9 5
53 12 5
54 9 6
55 11 5
56 10 7
57 10 4
58 17 9
59 7 6
60 19 11
61 9 6
62 4 4
63 13 11
64 13 7
65 13 6
66 8 7
67 14 8
68 14 5
69 16 10
70 18 13
71 16 21
72 30 8
73 6 3
74 9 5
75 35 17
76 7 4
77 29 18
78 24 14
79 10 9
80 4 4
81 24 12
82 23 12
83 41 30
84 39 49
85 28 12
86 14 8
87 9 5
88 30 16
89 32 17
90 23 12
91 16 5
92 8 4
93 9 5
94 13 16
95 38 23
96 32 16
97 21 16
98 6 5
99 2 4
100 11 7
101 19 11
102 10 4
103 6 5
104 10 4
105 9 5
106 31 11
107 17 15
108 30 18
109 17 10
110 40 19
111 7 3
112 28 7
113 10 10
114 19 9
115 28 12
116 32 15
117 7 4
118 4 5
119 8 5
120 5 5
121 7 7
122 9 5


927 6 3
928 10 5
929 10 8
930 6 3
931 7 7
932 10 10
933 11 6
934 11 4
935 23 10
936 16 6
937 24 13
938 26 15
939 10 6
940 4 4
941 23 10
942 22 2
943 7 7
944 7 5
945 7 4
946 13 7
947 10 6
948 12 7
949 7 7
950 8 5
951 5 3
952 9 4
953 7 9
954 13 8
955 11 6
956 14 6
957 10 5
958 28 8
959 16 8
960 11 5
961 6 4
962 15 8
963 11 8
964 18 7
965 32 7
966 16 7
967 16 9
968 8 3
969 14 5
970 12 5
971 21 9
972 31 13
973 9 6
974 7 4
975 14 4
976 7 5
977 7 1
978 19 11
979 19 10
980 20 13
981 31 15
982 8 5
983 8 4
984 6 4
985 5 3
986 8 5
987 1 3
988 13 4
989 0 6
990 7 3
991 5 11
992 9 4
993 24 11
994 2 2
995 13 6
996 8 5
997 10 6
998 25 13
999 10 4
1000 5 6
1001 6 7
1002 1 1
1003 19 6
1004 6 3
1005 34 11
1006 13 7
1007 18 6
1008 21 15
1009 9 7
1010 7 6
1011 4 4
1012 11 5
1013 3 1
1014 17 10
1015 17 8
1016 26 13
1017 6 3
1018 36 4
1019 24 9
1020 21 19
1021 4 2
1022 22 15
1023 6 5
1024 13 6
1025 8 7
1026 4 7
1027 18 9
1028 23 11
1029 11 1
1030 12 5
1031 14 6
1032 17 5
1033 8 6
1034 11 6
1035 13 4
1036 17

1763 19 12
1764 8 5
1765 6 6
1766 19 11
1767 19 12
1768 17 8
1769 14 9
1770 24 15
1771 17 8
1772 10 9
1773 27 11
1774 32 16
1775 26 15
1776 44 22
1777 7 4
1778 28 8
1779 16 10
1780 12 8
1781 34 12
1782 11 7
1783 25 33
1784 4 4
1785 17 16
1786 15 5
1787 58 5
1788 8 4
1789 11 7
1790 0 1
1791 7 7
1792 12 6
1793 9 5
1794 8 5
1795 12 6
1796 8 6
1797 20 11
1798 10 5
1799 5 5
1800 9 6
1801 21 8
1802 10 13
1803 32 29
1804 1 1
1805 27 14
1806 11 8
1807 3 3
1808 21 9
1809 21 9
1810 13 6
1811 5 3
1812 27 12
1813 5 3
1814 0 1
1815 6 4
1816 12 6
1817 8 4
1818 9 8
1819 7 5
1820 4 4
1821 17 4
1822 8 4
1823 27 12
1824 12 7
1825 20 8
1826 10 6
1827 17 5
1828 5 5
1829 6 5
1830 5 5
1831 84 19
1832 16 8
1833 16 13
1834 22 6
1835 14 8
1836 6 6
1837 15 10
1838 25 18
1839 22 7
1840 8 9
1841 15 8
1842 1 1
1843 9 2
1844 17 5
1845 17 6
1846 3 2
1847 15 7
1848 21 11
1849 14 8
1850 14 9
1851 34 19
1852 6 4
1853 9 22
1854 11 6
1855 17 8
1856 1 2
1857 0 1
1858 21 2
1859 15 7
1860 23 12
1861 4 4
1862 6 4
1863 23 16


2593 10 7
2594 9 4
2595 12 5
2596 10 8
2597 7 4
2598 7 4
2599 50 5
2600 8 5
2601 30 17
2602 12 8
2603 8 4
2604 14 8
2605 20 10
2606 22 12
2607 12 9
2608 31 16
2609 12 17
2610 21 5
2611 7 5
2612 11 7
2613 7 5
2614 20 4
2615 3 4
2616 7 5
2617 8 3
2618 5 2
2619 23 10
2620 3 1
2621 23 9
2622 4 2
2623 10 2
2624 5 9
2625 19 11
2626 18 8
2627 12 6
2628 21 14
2629 27 9
2630 3 2
2631 20 9
2632 26 7
2633 16 5
2634 49 14
2635 19 11
2636 11 4
2637 5 3
2638 27 14
2639 10 7
2640 9 2
2641 10 5
2642 10 5
2643 13 6
2644 7 7
2645 13 5
2646 19 14
2647 2 2
2648 8 7
2649 20 13
2650 10 9
2651 15 6
2652 3 2
2653 23 11
2654 24 15
2655 14 7
2656 11 8
2657 6 6
2658 33 14
2659 9 7
2660 15 7
2661 18 5
2662 3 2
2663 12 7
2664 9 5
2665 6 3
2666 21 13
2667 8 4
2668 14 10
2669 11 7
2670 12 3
2671 6 7
2672 14 5
2673 9 5
2674 9 5
2675 6 5
2676 18 4
2677 32 13
2678 6 3
2679 16 8
2680 15 8
2681 5 4
2682 6 4
2683 9 3
2684 13 7
2685 12 5
2686 16 10
2687 2 5
2688 7 4
2689 25 10
2690 32 12
2691 7 5
2692 2 5
2693 9 7
2694 13 

KeyboardInterrupt: 

In [15]:
#3256
with open('Paper_Dmpairs_ENG.pickle','wb') as f:
    pickle.dump(aligned,f)

# 以下實驗

In [19]:
ch_template = [['本','論文','主要','討論','機器','學習'],['機器人','在','現代'],['對','人類','很','重要'],['同時','電腦','也','很','重要'],['我','喜歡','論文']]
en_template = [['This','paper','mainly','discuss','machine','learning'],['Robots','are','important','to','people','nowadays','and','so','do','computers'],['I','like','paper']]

In [20]:
greedy(ch_template,en_template)

[['本論文主要討論機器學習', ' This paper mainly discuss machine learning'],
 ['對人類很重要', ' Robots are important to people nowadays and so do computers'],
 ['我喜歡論文', ' I like paper']]